# yFinance
Links for References used for this excercise: 
1. https://github.com/ranaroussi/yfinance
2. https://www.geeksforgeeks.org/python/multithreaded-download-of-yahoo-stock-history-with-python-yfinance/
3. For handling dates and time: 
    *  https://docs.python.org/3/library/datetime.html for calling date and time info
    *  https://stackoverflow.com/questions/32490629/getting-todays-date-in-yyyy-mm-dd-in-python for formatting the date info 
    *  https://www.geeksforgeeks.org/python/python-datetime-strptime-function/ for strptime changing Date and time into a string to be used in the filename
4. Saving dataframe to csv files https://www.datacamp.com/tutorial/save-as-csv-pandas-dataframe
5. Info about saving files into different directories in codespaces https://docs.github.com/en/codespaces/about-codespaces/deep-dive
6. Translate function for stripping out multiple characters from string filename https://www.geeksforgeeks.org/python/python-replace-multiple-characters-at-once/

## Problem 1: Data from yFinance

In [23]:
# import required modules 
import yfinance as yf
import time
import datetime
import pandas as pd
from pathlib import Path
import os

In [24]:
# define program start time and dates for use in calculations 
start= time.time()
today = datetime.date.today()
yesterday = today - datetime.timedelta(days=1)
fivedaysago = today - datetime.timedelta(days=5)

# Convert dates to string type data as is required by the yf.download function
strtoday = str(today)
strfivedaysago =str(fivedaysago)

#Checking the formats of the dates are correct
    #print (type(strtoday))
    #print (type(strfivedaysago))
print (f"Today's date is ",today)
print (f"Five days ago was ",fivedaysago)

Today's date is  2025-12-07
Five days ago was  2025-12-02


In [25]:
# Define a list of stocks to track
ticker_list =['META','AAPL','AMZN','NFLX','GOOG']

# Define path to root file
datadir = "./data/"# Problem 1: Data from yfinance


In [26]:
# Execute the yf.download operation and populating the data into a dataframe.  
data =yf.download( 
    start= strfivedaysago,
    end= strtoday,
    interval= '15m',
    tickers = ticker_list,
    threads=True,
    group_by='ticker',
    auto_adjust = True,
    )
type(data)

[*********************100%***********************]  5 of 5 completed


pandas.core.frame.DataFrame

In [27]:
# View the index
data.index

DatetimeIndex(['2025-12-02 14:30:00+00:00', '2025-12-02 14:45:00+00:00',
               '2025-12-02 15:00:00+00:00', '2025-12-02 15:15:00+00:00',
               '2025-12-02 15:30:00+00:00', '2025-12-02 15:45:00+00:00',
               '2025-12-02 16:00:00+00:00', '2025-12-02 16:15:00+00:00',
               '2025-12-02 16:30:00+00:00', '2025-12-02 16:45:00+00:00',
               ...
               '2025-12-05 18:30:00+00:00', '2025-12-05 18:45:00+00:00',
               '2025-12-05 19:00:00+00:00', '2025-12-05 19:15:00+00:00',
               '2025-12-05 19:30:00+00:00', '2025-12-05 19:45:00+00:00',
               '2025-12-05 20:00:00+00:00', '2025-12-05 20:15:00+00:00',
               '2025-12-05 20:30:00+00:00', '2025-12-05 20:45:00+00:00'],
              dtype='datetime64[ns, UTC]', name='Datetime', length=104, freq=None)

In [28]:
# view the data
data

Ticker                           AAPL                                      \
Price                            Open        High         Low       Close   
Datetime                                                                    
2025-12-02 14:30:00+00:00  283.000000  284.689911  282.730011  284.540009   
2025-12-02 14:45:00+00:00  284.510010  285.049988  283.079987  283.390015   
2025-12-02 15:00:00+00:00  283.394989  284.950012  283.260010  284.760010   
2025-12-02 15:15:00+00:00  284.750000  285.308990  284.480011  284.570007   
2025-12-02 15:30:00+00:00  284.540009  285.950012  284.149994  285.934998   
...                               ...         ...         ...         ...   
2025-12-05 19:45:00+00:00  278.880005  279.019989  278.728394  278.883209   
2025-12-05 20:00:00+00:00  278.890015  278.945007  278.595093  278.863403   
2025-12-05 20:15:00+00:00  278.859985  278.909912  278.540009  278.801392   
2025-12-05 20:30:00+00:00  278.799988  279.024994  278.790009  278.929993   
2025-12-05 20:45:00+00:00  278.920013  278.959991  278.190002  278.790009   

Ticker                                    META                          \
Price                       Volume        Open        High         Low   
Datetime                                                                 
2025-12-02 14:30:00+00:00  3529286  641.500000  643.879089  640.059998   
2025-12-02 14:45:00+00:00  1825716  640.309998  642.289978  638.070007   
2025-12-02 15:00:00+00:00  1169951  642.114990  643.765930  642.000000   
2025-12-02 15:15:00+00:00  1353220  642.719971  643.068787  641.580017   
2025-12-02 15:30:00+00:00  1491595  642.320007  643.200012  641.059998   
...                            ...         ...         ...         ...   
2025-12-05 19:45:00+00:00   505325  673.015015  673.424988  672.619995   
2025-12-05 20:00:00+00:00   486479  673.099976  673.169983  670.799988   
2025-12-05 20:15:00+00:00   482298  672.190002  673.419983  672.039978   
2025-12-05 20:30:00+00:00   581249  672.700012  673.799988  672.219971   
2025-12-05 20:45:00+00:00  2138221  673.270020  673.799988  671.719971   

Ticker                                         ...        AMZN              \
Price                           Close  Volume  ...        Open        High   
Datetime                                       ...                           
2025-12-02 14:30:00+00:00  640.309998  845882  ...  235.500000  235.539993   
2025-12-02 14:45:00+00:00  642.090027  450740  ...  234.910004  235.129898   
2025-12-02 15:00:00+00:00  642.750000  334310  ...  234.529999  235.770004   
2025-12-02 15:15:00+00:00  642.252502  242306  ...  235.669998  235.679993   
2025-12-02 15:30:00+00:00  643.150024  353622  ...  234.720001  235.350006   
...                               ...     ...  ...         ...         ...   
2025-12-05 19:45:00+00:00  673.099976  196405  ...  229.600006  229.729996   
2025-12-05 20:00:00+00:00  672.174988  258492  ...  229.699997  229.744995   
2025-12-05 20:15:00+00:00  672.710022  274763  ...  229.369995  229.610001   
2025-12-05 20:30:00+00:00  673.325012  381437  ...  229.250000  229.500000   
2025-12-05 20:45:00+00:00  673.460022  786080  ...  229.309998  229.639999   

Ticker                                                            NFLX  \
Price                             Low       Close   Volume        Open   
Datetime                                                                 
2025-12-02 14:30:00+00:00  234.050003  234.914993  3449594  108.500000   
2025-12-02 14:45:00+00:00  234.500000  234.520004  1087473  108.220001   
2025-12-02 15:00:00+00:00  234.410004  235.664001  1585231  108.474998   
2025-12-02 15:15:00+00:00  234.619995  234.720001  1230796  108.245003   
2025-12-02 15:30:00+00:00  234.539993  235.229996   865169  108.160004   
...                               ...         ...      ...         ...   
2025-12-05 19:45:00+00:00  229.445007  229.699997   303446  100.720001   
2025-12-05 20:00:00+00:00  229.365005  229.365005  

In [29]:
# Get date and time information for file name into the correct format
now = datetime.datetime.now()
filename = now.strftime("%Y%m%d-%H%M%S")

print (filename)

20251207-235150


In [30]:
# display the datadir
print (f"path is ", datadir)

# Write data to csv file. 
data.to_csv(datadir+filename+".csv", sep=',')

path is  ./data/


In [ ]:
# This segment allows for monitoring the time taken to complete the execution of the program. 
print ('The program takes ', time.time()-start,'seconds.')

The program takes  0.42041778564453125 seconds.


## Problem 2: Plotting Data

In [ ]:
# Problem 2: Plotting Data
# https://stackoverflow.com/questions/43074685/find-file-in-directory-with-the-highest-number-in-the-filename for filename with highest number, as this will be the latest created file

# Import the Rational Expressions module (re). 
import re
import matplotlib 

# List all files in the directory data 
list_of_files = os.listdir(datadir)
#print (list_of_files)

# list newest file amother way
#os.listdir('./data/')
list_of_files.sort(reverse = True)

#show this list
list_of_files

['20251207 233754.csv',
 '20251207 230315.csv',
 '20251207 230227.csv',
 '20251207 230146.csv',
 '20251207 230134.csv',
 '20251207 225354.csv',
 '20251207 224213.csv',
 '20251207 224142.csv',
 '20251207 222436.csv',
 '20251207 222244.csv',
 '20251207 222059.csv',
 '20251207 220622.csv',
 '20251207 220332.csv',
 '20251207 220307.csv',
 '20251207 220132.csv',
 '20251207 215510.csv',
 '20251207 214855.csv',
 '20251207 214845.csv',
 '20251207 213858.csv',
 '20251207 213657.csv',
 '20251207 212741.csv',
 '20251206 215647.csv',
 '20251126 211628.csv',
 '20251011 125240.csv',
 '20251010 221236.csv',
 '20251010 220006.csv',
 '20251010 215932.csv',
 '20251010 214240.csv',
 '20251010 213904.csv',
 '20251010 213843.csv',
 '20251010 211542.csv',
 '20251010 211004.csv',
 '20251010 210410.csv',
 '20251010 210356.csv',
 '20251010 210256.csv',
 '20251010 150510.csv',
 '20251010 150415.csv',
 '20251010 145916.csv',
 '20251010 145322.csv',
 '20251010 145034.csv',
 '20251010 144404.csv',
 '20251010 14433

In [ ]:
# show the latest file
list_of_files[0]

'20251207 233754.csv'

In [ ]:
# assign a variable name to the file selected as the latest file 
recent_file = list_of_files[0]
print(recent_file)

20251207 233754.csv


In [ ]:
# create a dataframe from the file selected as most recent
df = pd.read_csv(f'./data/{list_of_files[0]}', header=[0, 1])

df


,Ticker,Price,2025-12-02,2025-12-03,2025-12-04,2025-12-05
,META,Open,642.3400268554688,644.4099731445312,676.0,664.0
0,META,High,6.478700e+02,6.488500e+02,6.761000e+02,6.746900e+02
1,META,Low,6.380700e+02,6.375500e+02,6.600500e+02,6.623900e+02
2,META,Close,6.471000e+02,6.396000e+02,6.615300e+02,6.734200e+02
3,META,Volume,1.164090e+07,1.113430e+07,2.987460e+07,2.116690e+07
4,GOOG,Open,3.169000e+02,3.160300e+02,3.230450e+02,3.200000e+02
5,GOOG,High,3.186000e+02,3.220400e+02,3.230990e+02,3.238300e+02
6,GOOG,Low,3.142200e+02,3.142800e+02,3.155900e+02,3.199800e+02
7,GOOG,Close,3.160200e+02,3.206200e+02,3.183900e+02,3.220900e+02
8,GOOG,Volume,2.466820e+07,3.228510e+07,2.083160e+07,1.569370e+07


In [ ]:
# Create a dataframe that only contains the closing prices for each stock for each day
closing_prices = (df['Price']== "Close")
close_df = df.loc[closing_prices]
print (close_df)




ValueError: Cannot index with multidimensional key

In [ ]:
#Read a unique label for each unique stock label from column labelled 'Ticker'
unique_stock = df['Ticker'].unique()
print(unique_stock)

df1 = df[df['Ticker'] == unique_stock[0]] #Netflix
df2 = df[df['Ticker'] == unique_stock[1]] #Apple
df3 = df[df['Ticker'] == unique_stock[2]] #Amazon
df4 = df[df['Ticker'] == unique_stock[3]] #Meta
df5 = df[df['Ticker'] == unique_stock[4]] #Google

# print each individual stock
print (df1)
print (df2)
print (df3)
print (df4)
print (df5)


#date = yesterday
#print (date)

#plot
ax = matplotlib.pyplot.gca()
close_df.plot( x = '2025-10-09', y=['NFLX', 'AAPL', 'AMZN', 'META', 'GOOG'] , ax = ax )


['GOOG' 'META' 'AMZN' 'NFLX' 'AAPL']
  Ticker   Price    2025-12-02    2025-12-03    2025-12-04    2025-12-05
0   GOOG    Open  3.169000e+02  3.160300e+02  3.230450e+02  3.200000e+02
1   GOOG    High  3.186000e+02  3.220400e+02  3.230990e+02  3.238300e+02
2   GOOG     Low  3.142200e+02  3.142800e+02  3.155900e+02  3.199800e+02
3   GOOG   Close  3.160200e+02  3.206200e+02  3.183900e+02  3.220900e+02
4   GOOG  Volume  2.466820e+07  3.228510e+07  2.083160e+07  1.569370e+07
  Ticker   Price    2025-12-02    2025-12-03    2025-12-04    2025-12-05
5   META    Open  6.423400e+02  6.444100e+02  6.760000e+02  6.640000e+02
6   META    High  6.478700e+02  6.488500e+02  6.761000e+02  6.746900e+02
7   META     Low  6.380700e+02  6.375500e+02  6.600500e+02  6.623900e+02
8   META   Close  6.471000e+02  6.396000e+02  6.615300e+02  6.734200e+02
9   META  Volume  1.164090e+07  1.113430e+07  2.987460e+07  2.116690e+07
   Ticker   Price    2025-12-02    2025-12-03    2025-12-04    2025-12-05
10   AMZN    

AttributeError: module 'matplotlib' has no attribute 'pyplot'

In [ ]:
# define a plot for the closing price over the last 5 days
#ax=close_df.plot(kind = 'line', x = '2025-10-09', y = 'NFLX', marker = '.', c = 'red' , label = 'Netflix')
#pt.show()

# Create the plot_data() function
#def plot_data():
    #function goes here.
    #gibberish

#plot_data()

## End